In [10]:
# pip install pyomo
import pyomo.environ as pyomo
import pandas as pd
import numpy as np

In [2]:
SIZE = 365
GRID_BUY_PRICE = 0.3

INPUT_DEMAND = "../../data/demand.csv"

df = pd.read_csv(INPUT_DEMAND, index_col="t")
df

,demand
t,
0,201.644164
1,201.394918
2,201.985554
3,201.325092
4,200.392689
...,...
360,201.873386
361,200.359180
362,200.992932


In [3]:
# 1. Creating a model ##################
model = pyomo.ConcreteModel()

In [4]:
# 2. Setting Parameters ################
model.T = pyomo.Set(initialize=range(len(df)))
model.grid_buy_price = pyomo.Param(initialize=GRID_BUY_PRICE)
demand_dict = {}
for i, item in enumerate(df["demand"]):
    demand_dict[i] = item
model.demand = pyomo.Param(model.T, initialize=demand_dict)


In [5]:
# 3. Setting Variables #################
model.v_po_grid = pyomo.Var(model.T, domain=pyomo.PositiveReals)

In [6]:
# 4. Setting Constraints ###############


def demand_supply_rule(model, t):
    return model.demand[t] <= model.v_po_grid[t]


model.c_demand_supply = pyomo.Constraint(model.T, rule=demand_supply_rule)

In [7]:
# 5. Setting Object Function ###########
def objective_function(model):
    cost_arr = [0] * len(df)
    for t in model.T:
        cost_arr[t] = model.grid_buy_price * model.v_po_grid[t]
    return sum(cost_arr)


model.obj = pyomo.Objective(rule=objective_function)

In [8]:
# 6. Making a Solver and Running the model #
solver = pyomo.SolverFactory("cbc")
result = solver.solve(model)
result


{'Problem': [{'Name': 'unknown', 'Lower bound': 22009.53628, 'Upper bound': 22009.53628, 'Number of objectives': 1, 'Number of constraints': 365, 'Number of variables': 365, 'Number of nonzeros': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.0, 'Wallclock time': 0.01, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 0}}, 'Error rc': 0, 'Time': 0.0222318172454834}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [11]:
grid_power = np.zeros(len(df))
for t in model.T:
    grid_power[t] = model.v_po_grid[t]()
grid_power

array([201.64416, 201.39492, 201.98555, 201.32509, 200.39269, 200.71921,
       200.51185, 200.87285, 200.99202, 200.45958, 200.07116, 201.99014,
       200.04897, 200.3358 , 200.08825, 201.65064, 201.28303, 200.65562,
       201.15456, 201.618  , 201.1167 , 201.46754, 200.49645, 201.32469,
       200.00971, 201.20106, 200.36338, 200.93746, 201.99675, 201.55129,
       200.94047, 200.63714, 201.75854, 201.57645, 200.72813, 200.29569,
       201.55983, 200.59918, 200.21365, 200.62206, 200.07756, 201.09746,
       201.23266, 200.74207, 201.6425 , 201.85578, 200.09746, 200.91933,
       201.42053, 200.47849, 200.45282, 201.30816, 201.99487, 200.66833,
       201.13662, 200.20115, 200.63824, 201.25097, 201.73001, 201.13026,
       201.87526, 200.95907, 201.24389, 201.28018, 201.35323, 200.4509 ,
       201.29219, 200.84098, 200.10276, 201.12009, 200.25207, 200.31448,
       200.77623, 200.41779, 201.35505, 200.27115, 201.50669, 200.1593 ,
       200.72532, 201.97975, 200.89991, 200.83836, 

In [12]:
df = df.assign(grid_power=grid_power)
df

,demand,grid_power
t,,
0,201.644164,201.64416
1,201.394918,201.39492
2,201.985554,201.98555
3,201.325092,201.32509
4,200.392689,200.39269
...,...,...
360,201.873386,201.87339
361,200.359180,200.35918
362,200.992932,200.99293
